In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab Notebooks/master_thesis
!pip install git+https://github.com/feralvam/easse.git

In [ ]:
import easse

In [ ]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.8/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
#!pip install nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#!pip install pingouin
import numpy as np
import pandas as pd
#import pingouin as pg
from scipy.stats import zscore, spearmanr, pearsonr
from tqdm import tqdm

In [ ]:
import en_core_web_sm
en_core_web_sm.load()

In [ ]:
#import en_core_web_md
#en_core_web_md.load()
import spacy
spacy.cli.download('en_core_web_md')
spacy.load('en_core_web_md')

# import

In [ ]:
# From https://github.com/feralvam/metaeval-simplification/blob/main/notebooks/correlation.py
from itertools import combinations
import numpy as np
import pandas as pd
from scipy.stats import zscore, pearsonr, t
from scipy.stats.mstats import mquantiles
from sklearn.metrics import cohen_kappa_score
import sys


def _standardise_ratings(df, rater_id_cols, aspect_col):
    return df.groupby(rater_id_cols)[aspect_col].transform(lambda x: zscore(x))


def _simulate_two_annotators(ratings, num_ratings_annotatorA=1):
    ratings_shuffled = np.random.permutation(ratings)
    ratingA = np.mean(ratings_shuffled[:num_ratings_annotatorA])
    ratingB = np.mean(ratings_shuffled[num_ratings_annotatorA:])
    return [ratingA, ratingB]


def compute_inter_annotator_agreement(df_ratings, segment_id_cols, rater_id_cols, aspects,
                                      n_bins=5, use_quantiles=True, n_simulations=1000):
    iaa_per_aspect = {}
    for aspect in aspects:
        if f"{aspect}_zscore" not in df_ratings.columns:
            df_ratings[f"{aspect}_zscore"] = _standardise_ratings(df_ratings, rater_id_cols, aspect)
        df_scores = df_ratings[segment_id_cols + [f'{aspect}_zscore']]
        # Bin the data in n_bins
        if use_quantiles:  # equally-distributed
            _, bins_ranges = pd.qcut(df_scores[f'{aspect}_zscore'], q=n_bins, retbins=True)
        else:  # equally-spaced
            _, bins_ranges = pd.cut(df_scores[f'{aspect}_zscore'], bins=n_bins, retbins=True)
        kappa_values = []
        for _ in tqdm(range(n_simulations)):
            ratings_simulation = df_scores.groupby(segment_id_cols)[f'{aspect}_zscore'].apply(_simulate_two_annotators).to_list()
            raterA, raterB = zip(*ratings_simulation)
            kappa_values.append(cohen_kappa_score(np.digitize(raterA, bins_ranges), np.digitize(raterB, bins_ranges), weights='quadratic'))
        iaa_per_aspect[aspect] = (np.mean(kappa_values), np.std(kappa_values))
    return iaa_per_aspect


def compute_segment_scores(df_ratings, segment_id_cols, rater_id_cols, aspects):
    scores_cols = []
    for aspect in aspects:
        df_ratings[f"{aspect}_zscore"] = _standardise_ratings(df_ratings, rater_id_cols, aspect)
        scores_cols += [aspect, f"{aspect}_zscore"]
    df_segment_scores = df_ratings.groupby(segment_id_cols)[scores_cols].agg([np.mean])
    df_segment_scores.columns = [a for a, _ in df_segment_scores.columns]

    return df_segment_scores


def _select_pairs_in_group(group, min_score_difference=0):
    data = []
    for (system_a, score_a, zscore_a), (system_b, score_b, zscore_b) in combinations(group.values, 2):
        # select the pair if its absolute difference in DA scores is greater than 25
        if abs(score_a - score_b) > min_score_difference:
            data.append([system_a, score_a, zscore_a, system_b, score_b, zscore_b])
    df_selected_pairs = pd.DataFrame(data,
                                     columns=['system_a', "score_a", "zscore_a", "system_b", "score_b", "zscore_b"])
    return df_selected_pairs


def select_segment_pairs(df_human_scores, aspect, sentence_id_cols, system_id_cols):
    df_scores = df_human_scores.reset_index()
    cols_of_interest = system_id_cols + [aspect, f"{aspect}_zscore"]
    #cols_of_interest = [aspect, f"{aspect}_zscore"]
    selected_pairs = (df_scores.groupby(sentence_id_cols)[cols_of_interest].apply(_select_pairs_in_group)
                                .reset_index(level=1, drop=True)
                                .reset_index())
    return selected_pairs


def compute_relative_ranking_correlations(df_human_scores, df_metrics_scores, aspect,
                                          segment_id_cols, sentence_id_cols, system_id_cols,
                                          use_absolute_values=True, bootstrap_samples=1000):
    df_segment_pairs = select_segment_pairs(df_human_scores, aspect, sentence_id_cols, system_id_cols)

    df_all_scores = pd.merge(left=df_segment_pairs,
                             left_on=sentence_id_cols+['system_a'],
                             right=df_metrics_scores,
                             right_on=segment_id_cols)
    df_all_scores = pd.merge(left=df_all_scores,
                             left_on=sentence_id_cols+['system_b'],
                             right=df_metrics_scores,
                             right_on=segment_id_cols)

    metrics_names = [col for col in df_metrics_scores.columns if col not in segment_id_cols]

    # Compute the correlations
    print("Computing correlations...")
    correlations_data = []
    for metric in metrics_names:
        #corr = kendall_tau_wmt(df_all_scores[['zscore_a', 'zscore_b', f"{metric}_x", f"{metric}_y"]])
        #corr, pval = spearmanr(df_all_scores[['zscore_a', f"{metric}_x"]], df_all_scores[['zscore_b', f"{metric}_y"]], axis=None)
        corr, pval = spearmanr(df_all_scores[['zscore_a', 'zscore_b']], df_all_scores[[f"{metric}_x", f"{metric}_y"]], axis=None)
        if use_absolute_values:
            corr = abs(corr)
        correlations_data.append([metric, corr])
    df_correlations = pd.DataFrame(correlations_data, columns=['metric', 'corr'])

    # Bootstrap sampling
    print("Bootstrap sampling...")
    correlations_bootstrap_data = []
    for _ in range(bootstrap_samples):
        df_scores_sample = df_all_scores.sample(n=len(df_all_scores), replace=True)
        for metric in metrics_names:
            corr_sample = kendall_tau_wmt(df_scores_sample[['zscore_a', 'zscore_b', f"{metric}_x", f"{metric}_y"]])
            if use_absolute_values:
                corr_sample = abs(corr_sample)
            correlations_bootstrap_data.append([metric, corr_sample])
    df_bootstrap_correlations = pd.DataFrame(correlations_bootstrap_data, columns=['metric', 'corr'])

    # Compute 95% confidence intervals for each metric
    print("Computing 95% confidence intervals for each metric...")
    confidence_intervals = []
    for metric in metrics_names:
        metric_corr = df_bootstrap_correlations[df_bootstrap_correlations['metric'] == metric]['corr']
        # Equivalent to using the R function quantile with default type 7
        lower, upper = mquantiles(metric_corr, prob=[0.05, 0.95], alphap=1, betap=1)
        confidence_intervals.append(pd.Interval(left=lower, right=upper, closed='both'))
    df_correlations['conf_interval'] = confidence_intervals
    df_correlations.sort_values(by=['corr'], ascending=False, inplace=True, ignore_index=True)

    # Determine if the difference in performance is significant
    print("Determining if the difference in performance is significant...")
    metrics_names = df_correlations['metric'].to_list()
    significance_matrix = []
    winner_status = []
    for _, row_metric_a in df_correlations.iterrows():
        metric_a = row_metric_a['metric']
        ci_metric_a = row_metric_a['conf_interval']
        is_winner = True
        significance_row = []
        for _, row_metric_b in df_correlations.iterrows():
            metric_b = row_metric_b['metric']
            ci_metric_b = row_metric_b['conf_interval']
            # It's significant if confidence intervals do not overlap
            is_diff_stats_significant = ci_metric_a.left > ci_metric_b.right
            significance_row.append(is_diff_stats_significant)
            # Update winner status (not significantly outperformed by any other metric)
            if metric_b != metric_a:
                is_winner = is_winner and is_diff_stats_significant
        significance_matrix.append(significance_row)
        winner_status.append(is_winner)
    df_correlations['is_winner'] = winner_status
    df_significance = pd.DataFrame(np.array(significance_matrix), columns=metrics_names, index=metrics_names)

    return df_correlations, df_significance


def kendall_tau_wmt(df_scores):
    concordant = 0
    discordant = 0
    #print(df_scores)
    for _, (score_a, score_b, metric_a, metric_b) in df_scores.iterrows():
        if score_a < score_b:
            if metric_a < metric_b:
                concordant += 1
            else:
                discordant += 1
        elif score_a >= score_b:
            if metric_a <= metric_b:
                discordant += 1
            else:
                concordant += 1
    if (abs(concordant) + abs(discordant)) != 0:
        return (abs(concordant) - abs(discordant)) / (abs(concordant) + abs(discordant))
    else:
        print('concordant: ', concordant)
        print('discordant: ', discordant)
        print(df_scores)
        sys.exit(0)


def compute_direct_assessment_correlations(df_human_scores, df_metrics_scores, aspect, segment_id_cols,
                                           use_absolute_values=True):
    df_da_scores = df_human_scores.reset_index()
    cols_of_interest = segment_id_cols + [aspect, f"{aspect}_zscore"]
    df_da_scores = df_da_scores[cols_of_interest]
    df_all_scores = pd.merge(left=df_metrics_scores, right=df_da_scores, on=segment_id_cols)

    # Compute correlations metrics vs human scores
    print("Computing correlations...")
    metrics_names = [col for col in df_metrics_scores.columns if col not in segment_id_cols]
    correlations_data = []
    for metric in metrics_names:
        corr, p_value = pearsonr(df_all_scores[metric], df_all_scores[f'{aspect}_zscore'])
        if use_absolute_values:
            corr = abs(corr)
        correlations_data.append([metric, corr, p_value])
    df_correlations_metrics_human = pd.DataFrame(correlations_data, columns=['metric', 'corr', 'p_value'])
    df_correlations_metrics_human.sort_values(by=['corr'], ascending=False, inplace=True, ignore_index=True)

    # Compute correlations metrics vs metrics
    metrics_names = df_correlations_metrics_human['metric'].to_list()
    correlations_data = []
    for _, (metric_a, corr_metric_a, _) in df_correlations_metrics_human.iterrows():
        for _, (metric_b, corr_metric_b, _) in df_correlations_metrics_human.iterrows():
            corr_a_b, pvalue_a_b = pearsonr(df_all_scores[metric_a], df_all_scores[metric_b])
            if use_absolute_values:
                corr_a_b = abs(corr_a_b)
            correlations_data.append([metric_a, corr_metric_a,
                                      metric_b, corr_metric_b,
                                      corr_a_b, pvalue_a_b])
    df_correlations_metric_metric = pd.DataFrame(correlations_data,
                                                 columns=['metric_a', 'corr_metric_a',
                                                          'metric_b', 'corr_metric_b',
                                                          'corr_a_b', 'pvalue_a_b'])

    # Determine if the difference in performance is significant
    print("Determining if the difference in performance is significant...")
    significance_matrix = []
    winner_status = []
    for metric_a in metrics_names:
        df_correlations = df_correlations_metric_metric[df_correlations_metric_metric['metric_a'] == metric_a]
        is_winner = True
        significance_row = []
        for _, (_, corr_metric_a, metric_b, corr_metric_b, corr_a_b, _) in df_correlations.iterrows():
            p = np.nan
            if (metric_a != metric_b) and (corr_metric_a > corr_metric_b):
                _, p = williams_test(corr_metric_a, corr_metric_b, corr_a_b, len(df_human_scores))
            is_diff_stats_significant = p < 0.05
            if not is_diff_stats_significant:
                # we do not care about the exact values in cases where it's not significant
                p = np.nan
            significance_row.append(p)
            # Update winner status (not significantly outperformed by any other metric)
            if metric_a != metric_b:
                is_winner = is_winner and is_diff_stats_significant
        significance_matrix.append(significance_row)
        winner_status.append(is_winner)
    df_correlations_metrics_human['is_winner'] = winner_status
    df_significance = pd.DataFrame(np.array(significance_matrix), columns=metrics_names, index=metrics_names)

    return df_correlations_metrics_human, df_significance


# From https://github.com/inmoonlight/nlp-williams/blob/master/williams.py
def williams_test(r12, r13, r23, n):
    """The Williams test (Evan J. Williams. 1959. Regression Analysis, volume 14. Wiley, New York, USA)
    A test of whether the population correlation r12 equals the population correlation r13.
    Significant: p < 0.05
    Arguments:
        r12 (float): correlation between x1, x2
        r13 (float): correlation between x1, x3
        r23 (float): correlation between x2, x3
        n (int): size of the population
    Returns:
        t (float): Williams test result
        p (float): p-value of t-dist
    """
    assert (r12 >= r13), "r12 should be larger than r13"
    assert (n > 3), "n should be larger than 3"

    K = 1 - r12 ** 2 - r13 ** 2 - r23 ** 2 + 2 * r12 * r13 * r23
    denominator = np.sqrt(
        2 * K * (n - 1) / (n - 3) + (((r12 + r13) ** 2) / 4) * ((1 - r23) ** 3)
    )
    numerator = (r12 - r13) * np.sqrt((n - 1) * (1 + r23))
    p = 1 - t.cdf(numerator / denominator, df=n - 3)  # changed to n-3 on 30/11/14
    return t, p

In [ ]:
from easse.utils.constants import TEST_SETS_PATHS
from easse.utils.helpers import read_lines
import math
import scipy

def read_test_set(test_set, as_lists=False):
    orig_sents_path = TEST_SETS_PATHS[(test_set, "orig")]
    refs_sents_paths = TEST_SETS_PATHS[(test_set, "refs")]
    num_refs = len(refs_sents_paths)

    orig_sents = read_lines(orig_sents_path)
    refs_sents = [read_lines(ref_sents_path) for ref_sents_path in refs_sents_paths]

    if as_lists:
        return orig_sents, refs_sents

    fhs = [orig_sents] + refs_sents
    all_sent_id = []
    all_orig_sent = []
    all_ref_sents = []
    for sent_id, (orig_sent, *ref_sents) in enumerate(zip(*fhs), start=1):
        all_sent_id += [sent_id] * num_refs
        all_orig_sent += [orig_sent] * num_refs
        all_ref_sents += ref_sents
    return pd.DataFrame(
        list(zip(all_sent_id, all_orig_sent, all_ref_sents)),
        columns=["sent_id", "orig_sent", "ref_sent"],
    )


def collect_references(sent_ids, test_set_orig_sents, test_set_refs_sents, num_refs):
    orig_sents = []
    refs_sents = [[] for i in range(num_refs)]
    for sent_id in sent_ids:
        orig_sents.append(test_set_orig_sents[sent_id-1])
        for i, ref in enumerate(test_set_refs_sents):
            refs_sents[i].append(ref[sent_id-1])

    return orig_sents, refs_sents


def sigmoid(x):
    return 1 / (1 + math.exp(-x))

ModuleNotFoundError: ignored

In [ ]:
from easse.bleu import corpus_bleu
from easse.fkgl import corpus_fkgl
from easse.samsa import get_samsa_sentence_scores
from easse.sari import corpus_sari
#from easse.bertscore import corpus_bertscore
from bert_score import BERTScorer

In [ ]:
asset_orig, asset_refs = read_test_set("asset_test", as_lists=True)
turk_orig, turk_refs = read_test_set("turkcorpus_test", as_lists=True)
hsplit_orig, hsplit_refs = read_test_set("hsplit_test", as_lists=True)

# We create a dataset composed of all references together
all_orig = asset_orig
all_refs = asset_refs + turk_refs + hsplit_refs

EVAL_DATASETS = {
    "asset": (asset_orig, asset_refs, 10),  # (original, references, number of references)
    #"turk": (turk_orig, turk_refs, 8),
    #"hsplit": (hsplit_orig, hsplit_refs, 4),
    #"all": (all_orig, all_refs, 22)
}

# make human-written data as the same format of simplicity-DA

In [ ]:
def standardise_ratings(df, rater_id, aspect):
    return df.groupby(by=rater_id)[aspect].transform(lambda x: zscore(x))

In [ ]:
df_hg = pd.read_csv("./src/simplification_human_evaluations/questeval_simplification_likert_ratings.csv")
df_hg = df_hg[df_hg['simplification_type'] == 'human']
simplicity_zscore = standardise_ratings(df_hg[df_hg['aspect'] == 'simplicity'], rater_id='worker_id', aspect="rating")
meaning_zscore = standardise_ratings(df_hg[df_hg['aspect'] == 'meaning'], rater_id='worker_id', aspect="rating")
fluency_zscore = standardise_ratings(df_hg[df_hg['aspect'] == 'fluency'], rater_id='worker_id', aspect="rating")
df_hg_simp = df_hg[df_hg['aspect'] == 'simplicity'].copy().drop(columns=['aspect', 'rating'])
df_hg_simp['simplicity_zscore'] = simplicity_zscore
df_hg_simp['simplicity'] = df_hg[df_hg['aspect'] == 'simplicity']['rating']
df_hg_mean = df_hg[df_hg['aspect'] == 'meaning'].copy().drop(columns=['aspect', 'rating'])
df_hg_mean['meaning_zscore'] = meaning_zscore
df_hg_mean['meaning'] = df_hg[df_hg['aspect'] == 'meaning']['rating']
df_hg_flu = df_hg[df_hg['aspect'] == 'fluency'].copy().drop(columns=['aspect', 'rating'])
df_hg_flu['fluency_zscore'] = fluency_zscore
df_hg_flu['fluency'] = df_hg[df_hg['aspect'] == 'fluency']['rating']
df_hg_all = df_hg_simp.merge(df_hg_mean, on=['source', 'simplification', 'sentence_id', 'simplification_type', 'system_name', 'worker_id', 'references'])
df_hg_all = df_hg_all.merge(df_hg_flu, on=['source', 'simplification', 'sentence_id', 'simplification_type', 'system_name', 'worker_id', 'references'])
df_hg_all_grouped = df_hg_all.groupby(['sentence_id']).mean()
#df_hg_all_grouped['sentence_id'] = df_hg_all_grouped['sentence_id'].astype(int)
#df_hg_all_grouped['sentence_id'] = df_hg_all_grouped.index
df_hg_all_grouped.reset_index(inplace=True)
df_hg_all_ddup = df_hg_all.drop_duplicates(subset='sentence_id')
df_hg_all_grouped = df_hg_all_grouped.merge(df_hg_all_ddup.drop(columns=['worker_id', 'simplicity_zscore', 'meaning_zscore', 'fluency_zscore', 'simplicity', 'fluency', 'meaning']), on=['sentence_id'])
df_hg_all_grouped.rename(columns={'sentence_id': 'sent_id', 'system_name': 'sys_name', 'source': 'orig_sent', 'simplification': 'simp_sent'}, inplace=True)

<ipython-input-14-5fae8518b660>:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_hg_all_grouped = df_hg_all.groupby(['sentence_id']).mean()


In [ ]:
eval(df_hg_all_grouped.iloc[0]['references']) # attached references are from ASSET, because # of references is 10

['Since 2000, the recipient of the Kate Greenaway Medal has also won the £5000 Colin Mears Award.',
 'Since 2000, the winner of the Kate Greenaway Medal also receives the Colin Mears Award.  The value of the prize is £5000.',
 'Since 2000, the person that receives the Kate Greenaway Medal has also been presented with the Colin Mears Award that has a value of £5000.',
 'Since 2000, the Kate Greenaway Medal winner has also been presented with the Colin Mears Award, valued £5000.',
 'Since 2000, the winner of the Kate Greenaway Medal won the  Colin Mears Award to the value of £5000.',
 'Whoever wins the Kate Greenway Medal also gets The Colin Mears award which is L5000.',
 'Since 2000, the recipient of the Kate Greenaway Medal will also receive the Colin Mears Awad which worth 5000 pounds.',
 'Since 2000, the recipient of the Kate Greenaway Medal has also been given the Colin Mears Award.',
 'The winner of the Kate Greenaway Medal has also been given the Colin Mears Award (valued at £5000

# get df_metrics_segment

In [ ]:
lowercase = False  # case-insensitive
tokenizer = "moses"
bertscore_rescale = BERTScorer(lang="en", rescale_with_baseline=True)

In [ ]:
df_hg_all_grouped

In [ ]:
for _, row in tqdm(df_hg_all_grouped.iterrows()):
    orig_sents = [row['source']]
    ref_sents = [[i] for i in eval(row['references'])]
    simp_sents = [row['simplification']]
    print(orig_sents)
    print(ref_sents)
    print(simp_sents)
    print()

In [ ]:
metrics = []
test_set = "asset"
for _, row in tqdm(df_hg_all_grouped.iterrows()):
    orig_sents = [row['orig_sent']]
    ref_sents = [[i] for i in eval(row['references'])]
    simp_sents = [row['simp_sent']]

    # BLEU
    bleu_sys_refs = corpus_bleu(
        [row["simp_sent"]],
        ref_sents,
        smooth_method="floor",
        tokenizer=tokenizer,
        lowercase=lowercase,
        effective_order=True,
    )

    # SARI
    sari_score = corpus_sari(
        orig_sents,
        [row["simp_sent"]],
        ref_sents,
        tokenizer=tokenizer,
        lowercase=lowercase,
        use_f1_for_deletion=False,
    )

    # # iBLEU (alpha = 0.9)
    # bleu_sys_orig = corpus_bleu(
    #     [row["simp_sent"]],
    #     [orig_sents],
    #     force=True,
    #     tokenizer=tokenizer,
    #     lowercase=lowercase,
    # )
    # ibleu_score = 0.9 * bleu_sys_refs - (1 - 0.9) * bleu_sys_orig

    # # Avg. of BLEU and SARI
    # amean_bleu_sari = np.mean([bleu_sys_refs, sari_score])
    # gmean_bleu_sari = scipy.stats.gmean([bleu_sys_refs, sari_score])

    # Flesch
    fkgl_sys = corpus_fkgl([row["simp_sent"]], tokenizer=tokenizer)

    # # FKBLEU
    # fkgl_orig = corpus_fkgl(orig_sents, tokenizer=tokenizer)
    # fk_diff = sigmoid(fkgl_sys - fkgl_orig)
    # fkbleu_score = ibleu_score * fk_diff

    # BERTScore
    ref_sents = [ref for [ref] in ref_sents]
    bertscore_rescale_scores = bertscore_rescale.score([row["simp_sent"]], [ref_sents])
    #bertscores = corpus_bertscore([
    #    row["simp_sent"]],
    #    ref_sents,
    #    tokenizer=tokenizer,
    #    lowercase=lowercase
    #)

    metrics.append(
        {
            "sent_id": row["sent_id"],
            "sys_name": row["sys_name"],
            "test_set": test_set,
            "bleu": bleu_sys_refs,
            "sari": sari_score,
            "ibleu": ibleu_score,
            "amean_bleu_sari": amean_bleu_sari,
            "gmean_bleu_sari": gmean_bleu_sari,
            "fkgl": fkgl_sys,
            "fkbleu": fkbleu_score,
            "bertscore_P": bertscore_rescale_scores[0].cpu().item(),
            "bertscore_R": bertscore_rescale_scores[1].cpu().item(),
            "bertscore_F1": bertscore_rescale_scores[2].cpu().item(),
            #"bertscore_P": bertscores[0],
            #"bertscore_R": bertscores[1],
            #"bertscore_F1": bertscores[2],
        }
    )

df_metrics_segment = pd.DataFrame(metrics)

In [ ]:
samsa_scores = get_samsa_sentence_scores(
    df_hg_all_grouped["orig_sent"],
    df_hg_all_grouped["simp_sent"],
    tokenizer=tokenizer,
    lowercase=lowercase,
)

# Since SAMSA is reference-less, this reformating is only done so that it can appear in thae same dataframe as the other metrics
df_metrics_segment["samsa"] = [
    s for s in samsa_scores for _ in range(len(EVAL_DATASETS))
]

In [ ]:
df_metrics_segment['amean_bleu_samsa'] = np.mean(df_metrics_segment[['bleu', 'samsa']], axis=1)
df_metrics_segment['amean_sari_samsa'] = np.mean(df_metrics_segment[['sari', 'samsa']], axis=1)
df_metrics_segment['gmean_bleu_samsa'] = scipy.stats.gmean(df_metrics_segment[['bleu', 'samsa']], axis=1)
df_metrics_segment['gmean_sari_samsa'] = scipy.stats.gmean(df_metrics_segment[['sari', 'samsa']], axis=1)

In [ ]:
import pickle
with open('./src/simplification_human_evaluations/df_metrics_segment_hg.pickle', 'wb') as f:
    pickle.dump(df_metrics_segment, f)

# compute correlations

In [ ]:
import pickle
with open('./src/simplification_human_evaluations/df_metrics_segment_hg.pickle', 'rb') as f:
    df_metrics_segment = pickle.load(f)

## direct assessment

In [ ]:
def compute_direct_correlations(df_benchmark, df_segment_metrics, aspects, test_sets, segment_id_cols):
    results = {}
    for aspect in aspects:
        smed = df_benchmark['simplicity_zscore'].median()
        shind = df_benchmark['simplicity_zscore'] > smed
        slind = df_benchmark['simplicity_zscore'] <= smed
        fmed = df_benchmark['fluency_zscore'].median()
        fhind = df_benchmark['fluency_zscore'] > fmed
        flind = df_benchmark['fluency_zscore'] <= fmed
        mmed = df_benchmark['meaning_zscore'].median()
        mhind = df_benchmark['meaning_zscore'] > mmed
        mlind = df_benchmark['meaning_zscore'] <= mmed
        fmhind = (df_benchmark['fluency_zscore'] > fmed) & (df_benchmark['meaning_zscore'] > mmed)
        fmlind = (df_benchmark['fluency_zscore'] <= fmed) & (df_benchmark['meaning_zscore'] <= mmed)
        df_scores_slow = df_benchmark[slind]
        df_scores_shigh = df_benchmark[shind]
        df_scores_flow = df_benchmark[flind]
        df_scores_fhigh = df_benchmark[fhind]
        df_scores_mlow = df_benchmark[mlind]
        df_scores_mhigh = df_benchmark[mhind]
        df_scores_fmlow = df_benchmark[fmlind]
        df_scores_fmhigh = df_benchmark[fmhind]

        print(f"{aspect}: simp_high ({len(df_scores_shigh)}) - simp_low ({len(df_scores_slow)}) - flu_low ({len(df_scores_flow)}) - flu_high ({len(df_scores_fhigh)}) - mean_low ({len(df_scores_mlow)}) - mean_high ({len(df_scores_mhigh)}) - fm_low ({len(df_scores_fmlow)}) - fm_high ({len(df_scores_fmhigh)}) - All ({len(df_benchmark)})")
        for quality, df_scores in {'simp_low': df_scores_slow, 'simp_high': df_scores_shigh, 'flu_low': df_scores_flow, 'flu_high': df_scores_fhigh, 'mean_low': df_scores_mlow, 'mean_high': df_scores_mhigh, 'fm_low': df_scores_fmlow, 'fm_high': df_scores_fmhigh, 'all': df_benchmark}.items():
            for test_set in test_sets:
                print(f"Computing for {quality} scores - {test_set} references")
                df_metrics = df_segment_metrics[df_segment_metrics.test_set==test_set].drop(columns=['test_set'])
                results[(quality, aspect, test_set)] = compute_direct_assessment_correlations(
                    df_scores,
                    df_metrics,
                    aspect,
                    segment_id_cols=segment_id_cols,
                    use_absolute_values=False
                )
                print()
    return results

In [ ]:
METRICS_TO_ANALYSE = ['bleu', 'sari', 'fkgl', 'fkbleu', 'ibleu',
                      'amean_bleu_sari',
                      'gmean_bleu_sari',
                      'bertscore_P', 'bertscore_R', 'bertscore_F1']

In [ ]:
EVAL_DATASETS = ['asset']

In [ ]:
df_hg_all_grouped.rename(columns={'sentence_id': 'sent_id', 'system_name': 'sys_name'}, inplace=True)

In [ ]:
results_simplicity = compute_direct_correlations(
                        df_hg_all_grouped,
                        df_metrics_segment,
                        aspects=['simplicity'],
                        test_sets=EVAL_DATASETS,
                        segment_id_cols=['sent_id','sys_name']
                    )

simplicity: simp_high (50) - simp_low (50) - flu_low (50) - flu_high (50) - mean_low (50) - mean_high (50) - fm_low (35) - fm_high (35) - All (100)
Computing for simp_low scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for simp_high scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for flu_low scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for flu_high scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for mean_low scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for mean_high scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for fm_

In [ ]:
results_simplicity[('all', 'simplicity', 'asset')][0]

,metric,corr,p_value,is_winner
0,bertscore_P,0.417633,0.000015,False
1,gmean_bleu_sari,0.415641,0.000017,False
2,amean_bleu_sari,0.407919,0.000025,False
3,ibleu,0.396765,0.000044,False
4,bertscore_F1,0.393838,0.000050,False
5,sari,0.390478,0.000059,False
6,bertscore_R,0.374249,0.000125,False
7,bleu,0.349962,0.000358,False
8,fkbleu,-0.146599,0.145549,False
9,fkgl,-0.353133,0.000314,False


## relative ranking

In [ ]:
def compute_relative_ranking_correlations(df_human_scores, df_metrics_scores, aspect, segment_id_cols,
                                           use_absolute_values=True):
    df_da_scores = df_human_scores.reset_index()
    cols_of_interest = segment_id_cols + [aspect, f"{aspect}_zscore"]
    df_da_scores = df_da_scores[cols_of_interest]
    df_all_scores = pd.merge(left=df_metrics_scores, right=df_da_scores, on=segment_id_cols)

    # Compute correlations metrics vs human scores
    print("Computing correlations...")
    metrics_names = [col for col in df_metrics_scores.columns if col not in segment_id_cols]
    correlations_data = []
    for metric in metrics_names:
        corr, p_value = spearmanr(df_all_scores[metric], df_all_scores[f'{aspect}_zscore'])
        if use_absolute_values:
            corr = abs(corr)
        correlations_data.append([metric, corr, p_value])
    df_correlations_metrics_human = pd.DataFrame(correlations_data, columns=['metric', 'corr', 'p_value'])
    df_correlations_metrics_human.sort_values(by=['corr'], ascending=False, inplace=True, ignore_index=True)

    # Compute correlations metrics vs metrics
    metrics_names = df_correlations_metrics_human['metric'].to_list()
    correlations_data = []
    for _, (metric_a, corr_metric_a, _) in df_correlations_metrics_human.iterrows():
        for _, (metric_b, corr_metric_b, _) in df_correlations_metrics_human.iterrows():
            corr_a_b, pvalue_a_b = pearsonr(df_all_scores[metric_a], df_all_scores[metric_b])
            if use_absolute_values:
                corr_a_b = abs(corr_a_b)
            correlations_data.append([metric_a, corr_metric_a,
                                      metric_b, corr_metric_b,
                                      corr_a_b, pvalue_a_b])
    df_correlations_metric_metric = pd.DataFrame(correlations_data,
                                                 columns=['metric_a', 'corr_metric_a',
                                                          'metric_b', 'corr_metric_b',
                                                          'corr_a_b', 'pvalue_a_b'])

    # Determine if the difference in performance is significant
    print("Determining if the difference in performance is significant...")
    significance_matrix = []
    winner_status = []
    for metric_a in metrics_names:
        df_correlations = df_correlations_metric_metric[df_correlations_metric_metric['metric_a'] == metric_a]
        is_winner = True
        significance_row = []
        for _, (_, corr_metric_a, metric_b, corr_metric_b, corr_a_b, _) in df_correlations.iterrows():
            p = np.nan
            if (metric_a != metric_b) and (corr_metric_a > corr_metric_b):
                _, p = williams_test(corr_metric_a, corr_metric_b, corr_a_b, len(df_human_scores))
            is_diff_stats_significant = p < 0.05
            if not is_diff_stats_significant:
                # we do not care about the exact values in cases where it's not significant
                p = np.nan
            significance_row.append(p)
            # Update winner status (not significantly outperformed by any other metric)
            if metric_a != metric_b:
                is_winner = is_winner and is_diff_stats_significant
        significance_matrix.append(significance_row)
        winner_status.append(is_winner)
    df_correlations_metrics_human['is_winner'] = winner_status
    df_significance = pd.DataFrame(np.array(significance_matrix), columns=metrics_names, index=metrics_names)

    return df_correlations_metrics_human, df_significance



In [ ]:
def compute_ranking_correlations(df_benchmark, df_segment_metrics, aspects, test_sets, segment_id_cols):
    results = {}
    for aspect in aspects:
        smed = df_benchmark['simplicity_zscore'].median()
        shind = df_benchmark['simplicity_zscore'] > smed
        slind = df_benchmark['simplicity_zscore'] <= smed
        fmed = df_benchmark['fluency_zscore'].median()
        fhind = df_benchmark['fluency_zscore'] > fmed
        flind = df_benchmark['fluency_zscore'] <= fmed
        mmed = df_benchmark['meaning_zscore'].median()
        mhind = df_benchmark['meaning_zscore'] > mmed
        mlind = df_benchmark['meaning_zscore'] <= mmed
        fmhind = (df_benchmark['fluency_zscore'] > fmed) & (df_benchmark['meaning_zscore'] > mmed)
        fmlind = (df_benchmark['fluency_zscore'] <= fmed) & (df_benchmark['meaning_zscore'] <= mmed)
        df_scores_slow = df_benchmark[slind]
        df_scores_shigh = df_benchmark[shind]
        df_scores_flow = df_benchmark[flind]
        df_scores_fhigh = df_benchmark[fhind]
        df_scores_mlow = df_benchmark[mlind]
        df_scores_mhigh = df_benchmark[mhind]
        df_scores_fmlow = df_benchmark[fmlind]
        df_scores_fmhigh = df_benchmark[fmhind]

        print(f"{aspect}: simp_high ({len(df_scores_shigh)}) - simp_low ({len(df_scores_slow)}) - flu_low ({len(df_scores_flow)}) - flu_high ({len(df_scores_fhigh)}) - mean_low ({len(df_scores_mlow)}) - mean_high ({len(df_scores_mhigh)}) - fm_low ({len(df_scores_fmlow)}) - fm_high ({len(df_scores_fmhigh)}) - All ({len(df_benchmark)})")
        for quality, df_scores in {'simp_low': df_scores_slow, 'simp_high': df_scores_shigh, 'flu_low': df_scores_flow, 'flu_high': df_scores_fhigh, 'mean_low': df_scores_mlow, 'mean_high': df_scores_mhigh, 'fm_low': df_scores_fmlow, 'fm_high': df_scores_fmhigh, 'all': df_benchmark}.items():
            for test_set in test_sets:
                print(f"Computing for {quality} scores - {test_set} references")
                df_metrics = df_segment_metrics[df_segment_metrics.test_set==test_set].drop(columns=['test_set'])
                results[(quality, aspect, test_set)] = compute_relative_ranking_correlations(
                    df_human_scores=df_scores,
                    df_metrics_scores=df_metrics,
                    aspect=aspect,
                    segment_id_cols=segment_id_cols,
                    #sentence_id_cols=['sent_id'],
                    #system_id_cols=['sys_name'],
                    use_absolute_values=False,
                )
                print()
    return results

In [ ]:
METRICS_TO_ANALYSE = ['bleu', 'sari', 'fkgl', 'fkbleu', 'ibleu',
                      'amean_bleu_sari',
                      'gmean_bleu_sari',
                      'bertscore_P', 'bertscore_R', 'bertscore_F1']
EVAL_DATASETS = ['asset']
df_hg_all_grouped.rename(columns={'sentence_id': 'sent_id', 'system_name': 'sys_name'}, inplace=True)

In [ ]:
results_simplicity = compute_ranking_correlations(
                        df_hg_all_grouped,
                        df_metrics_segment,
                        aspects=['simplicity'],
                        test_sets=['asset'],
                        segment_id_cols=['sent_id','sys_name']
                    )

simplicity: simp_high (50) - simp_low (50) - flu_low (50) - flu_high (50) - mean_low (50) - mean_high (50) - fm_low (35) - fm_high (35) - All (100)
Computing for simp_low scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for simp_high scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for flu_low scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for flu_high scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for mean_low scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for mean_high scores - asset references
Computing correlations...
Determining if the difference in performance is significant...

Computing for fm_

In [ ]:
results_simplicity[('all', 'simplicity', 'asset')][0]

,metric,corr,p_value,is_winner
0,bertscore_R,0.401587,0.000035,False
1,bertscore_F1,0.393353,0.000052,False
2,gmean_bleu_sari,0.391011,0.000058,False
3,bertscore_P,0.387352,0.000069,False
4,amean_bleu_sari,0.385191,0.000076,False
5,sari,0.373777,0.000128,False
6,ibleu,0.346307,0.000417,False
7,bleu,0.312440,0.001552,False
8,fkbleu,-0.299862,0.002438,False
9,fkgl,-0.359319,0.000241,False
